In [ ]:
"""
Copyright © 2021-2022 The Johns Hopkins University Applied Physics Laboratory LLC

Permission is hereby granted, free of charge, to any person obtaining a copy 
of this software and associated documentation files (the “Software”), to 
deal in the Software without restriction, including without limitation the 
rights to use, copy, modify, merge, publish, distribute, sublicense, and/or 
sell copies of the Software, and to permit persons to whom the Software is 
furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in 
all copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED “AS IS”, WITHOUT WARRANTY OF ANY KIND, EXPRESS OR 
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, 
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE 
AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, 
WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR 
IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE.
"""

In [ ]:
# Import necessary modules
import json
from pathlib import Path

import pandas as pd
from l2metrics.report import MetricsReport
from tabulate import tabulate

pd.options.display.float_format = "{:,.2f}".format
%matplotlib ipympl


In [ ]:
# Configure metrics report
kwargs = {}
kwargs["log_dir"] = "multi_task"
kwargs["variant_mode"] = "aware"
kwargs["ste_averaging_method"] = "metrics"
kwargs["perf_measure"] = "performance"
kwargs["aggregation_method"] = "mean"
kwargs["maintenance_method"] = "both"
kwargs["transfer_method"] = "both"
kwargs["normalization_method"] = "task"
kwargs["smoothing_method"] = "flat"
kwargs["window_length"] = None
kwargs["clamp_outliers"] = True
kwargs["data_range_file"] = None  # "data_range.json"
kwargs["show_eval_lines"] = True
kwargs["unit"] = "steps"
kwargs["do_plot"] = True
kwargs["do_save"] = False
kwargs["do_save_settings"] = False

output_dir = Path("results")
output = "ll_metrics"

# Create output directory if it doesn't exist
if kwargs["do_save"] or kwargs["do_save_settings"]:
    output_dir.mkdir(parents=True, exist_ok=True)

# Load data range data for normalization and standardize names to lowercase
if kwargs["data_range_file"]:
    with open(kwargs["data_range_file"]) as config_json:
        data_range = json.load(config_json)
        data_range = {key.lower(): val for key, val in data_range.items()}
else:
    data_range = None
kwargs["data_range"] = data_range


In [ ]:
# Initialize metrics report
report = MetricsReport(**kwargs)

In [ ]:
# Add noise to log data
# report.add_noise(mean=0.0, std=1.0)

In [ ]:
# Calculate metrics in order of their addition to the metrics list.
report.calculate()

In [ ]:
# Print log summary by LX and EX counts
log_summary_df = report.log_summary()
display(log_summary_df)

print(f"\nTotal number of experiences: {report._log_data.shape[0]}")

In [ ]:
print("Lifetime metrics:")
display(report.lifetime_metrics_df)

print("\nTask metrics:")
display(report.task_metrics_df)

In [ ]:
# Plot raw learning curves for scenario
if kwargs["do_plot"]:
    report.plot(
        plot_types="raw",
        save=kwargs["do_save"],
        output_dir=str(output_dir),
    )


In [ ]:
# Plot processed learning curves for scenario
if kwargs["do_plot"]:
    report.plot(
        plot_types="lb",
        save=kwargs["do_save"],
        show_eval_lines=kwargs["show_eval_lines"],
        output_dir=str(output_dir),
    )


In [ ]:
# Plot evaluation lines for scenario
if kwargs["do_plot"]:
    report.plot(
        plot_types="eb",
        save=kwargs["do_save"],
        output_dir=str(output_dir),
    )


In [ ]:
# Plot tasks and their corresponding STE
if kwargs["do_plot"]:
    report.plot(plot_types="ste", save=kwargs["do_save"], output_dir=str(output_dir))


In [ ]:
# Save data to feather file
if kwargs["do_save"]:
    report.save_data(output_dir=str(output_dir), filename=output)

In [ ]:
# Print block summary
print(tabulate(report.block_info, headers="keys", tablefmt="psql"))

In [ ]:
# Print metrics per regime
print(
    tabulate(
        report.regime_metrics_df.fillna("N/A"),
        headers="keys",
        tablefmt="psql",
        floatfmt=".2f",
    )
)